In [2]:
# usual imports
import os
import sys
import time
import glob
import datetime
import sqlite3
import pandas as pd
import numpy as np # get it at: http://numpy.scipy.org/
# path to the Million Song Dataset subset (uncompressed)
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_subset_path='./data/MSD/MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path' # sanity check
# path to the Million Song Dataset code
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_code_path='./data/MSD/MSongsDB'
assert os.path.isdir(msd_code_path),'wrong path' # sanity check
# we add some paths to python so we can import MSD code
# Ubuntu: you can change the environment variable PYTHONPATH
# in your .bashrc file so you do not have to type these lines
sys.path.append(os.path.join(msd_code_path,'PythonSrc') )

taste_profile_data_path="./data/TinyEchoNestTasteProfileSubset.csv"
assert os.path.isfile(taste_profile_data_path)

# imports specific to the MSD
import hdf5_getters as GETTERS

from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# Get Data

In [3]:
def reduce_taste_subset(path='./data/FullEchoNestTasteProfileSubset.txt', 
                        to_path='./data/TeenyTinyEchoNestTasteProfileSubset.csv', downsample=0.001):
    data = pd.read_csv(path, sep="\t", header=None)
    data.columns = ['user', 'song', 'play_count']
    data.astype({'user': np.str, 'song': np.str, 'play_count': np.int32})
    data.sample(frac=downsample).to_csv(to_path, index=False)
# the following function simply gives us a nice string for
# a time lag in seconds
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

In [4]:
data = pd.read_csv(taste_profile_data_path)

In [5]:
data.head()

,user,song,play_count
0,293073e399f3d3b3dc38b8043175d23902645f4c,SORGAPU12A8AE4585E,1
1,b5f15729a33087a849df1747f99adf699dc154d9,SOJYCEU12A6701FBAA,1
2,e35f2113adc67b2ce5372a2eff4115f5d38133c7,SOBEGGQ12A8C1341E0,7
3,e7d091018fede4482d03eddb414cd7e245cd476a,SOVVHRL12A6D4F6BF7,1
4,d7de5fcb8f1fa64e4eaab85a51f7e59d0c8040d7,SOQWZHE12A67AE10FF,5


In [5]:
song_id = data.iloc[1]['song']
print(data.shape)

(483736, 3)


In [ ]:
a = "e45503e1d7c3c88fcea0da0947292afdbb26d27c"


### To check if the data we are using is also in the MSD
Running the cell to get all valid_zone_ids showed that all songs in data are valid songs in the MSD

In [6]:
# Get the song ids which are also in the database. 
valid_song_ids = []

for song_id in data['song'].values:
    # let's redo all this work in SQLite in a few seconds
    t1 = time.time()
    # connect to database to get the metadata from MSD
    conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                        'subset_track_metadata.db'))

    q = "SELECT DISTINCT song_id,artist_id,artist_name,Count(track_id) FROM songs"
    q += " WHERE song_id='" + song_id + "'"
    res = conn.execute(q)
    res = res.fetchall()
    conn.close()
    t2 = time.time()
    if res[0] is not None:
        valid_song_ids.append(song_id)


In [7]:
len(valid_song_ids)

48374

# Fit model

In [6]:


def tune_ALS(train_data, validation_data, maxIter, regParams, ranks):
    """
    grid search function to select the best model based on RMSE of
    validation data
    Parameters
    ----------
    train_data: spark DF with columns ['user', 'song', 'play_count']
    
    validation_data: spark DF with columns ['user', 'song', 'play_count']
    
    maxIter: int, max number of learning iterations
    
    regParams: list of float, one dimension of hyper-param tuning grid
    
    ranks: list of float, one dimension of hyper-param tuning grid
    
    Return
    ------
    The best fitted ALS model with lowest RMSE score on validation data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in regParams:
            # get ALS model
            # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
            als = ALS(userCol="user", itemCol="song", ratingCol="play_count", implicitPrefs=True, alpha=40).setMaxIter(maxIter).setRank(rank).setRegParam(reg).setColdStartStrategy("drop")
            # train ALS model
            model = als.fit(train_data)
            # evaluate the model by computing the RMSE on the validation data
            predictions = model.transform(validation_data)  # give Nan to data that hasn't been seen before. cold_start drops those
            evaluator = RegressionEvaluator(metricName="rmse",
                                            labelCol="play_count",
                                            predictionCol="prediction")
            rmse = evaluator.evaluate(predictions)
            print('{} latent factors and regularization = {}: '
                  'validation RMSE is {}'.format(rank, reg, rmse))
            if rmse < min_error:
                min_error = rmse
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and '
          'regularization = {}'.format(best_rank, best_regularization))
    return best_model

In [7]:
user_mapping = {}
last_user = 0
song_mapping = {}
last_song = 0
for idx, row in data.iterrows():
    if row["user"] not in user_mapping:
        user_mapping[row["user"]] = last_user
        last_user += 1
    if row["song"] not in song_mapping:
        song_mapping[row["song"]] = last_song
        last_song += 1


In [8]:
prep_data = data.copy()
prep_data["user"] = prep_data["user"].map(lambda x: user_mapping[x])
prep_data["song"] = prep_data["song"].map(lambda x: song_mapping[x])

In [9]:
split_frac = 0.7
train_data = prep_data.iloc[:int(prep_data.shape[0] * split_frac)]
val_data = prep_data.iloc[int(prep_data.shape[0] * split_frac):]

In [13]:
spark_data = spark.createDataFrame(prep_data)
(training, val) = spark_data.randomSplit([0.8, 0.2])


In [11]:
train_data.head()

,user,song,play_count
0,0,0,1
1,1,1,1
2,2,2,7
3,3,3,1
4,4,4,5


In [14]:
model = tune_ALS(training, val, 20, [1e-3, 1e-2, 1e-1, 1, 2], [50])

50 latent factors and regularization = 0.001: validation RMSE is 5.62476611477
50 latent factors and regularization = 0.01: validation RMSE is 5.62476757102
50 latent factors and regularization = 0.1: validation RMSE is 5.62477833389
50 latent factors and regularization = 1: validation RMSE is 5.62474339512
50 latent factors and regularization = 2: validation RMSE is 5.6246331639

The best model has 50 latent factors and regularization = 2


In [75]:
model

ALS_dd3b6079bc21